# p-median

## 필요한 모듈 설치 및 라이브러리 불러오기

In [1]:
!pip install pulp
!pip install haversine

In [2]:
# data handling
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings(action = "ignore")

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 좌표 단위 변환
from functools import partial
from pyproj import Proj, transform

# p-median
from pulp import *

# haversine
from haversine import haversine

In [3]:
random.seed(0)

shelter = pd.read_csv("./data/강서구_경로당_좌표.csv", encoding="cp949")

path='./data/senior'
folder = os.listdir(path)
people = pd.DataFrame()
for file in folder:
    df = pd.read_csv(f"./data/senior/{file}", encoding="cp949")
    people = pd.concat([people, df], axis=0)
    
people.reset_index(drop=True, inplace=True)

people.columns = ["x", "y"]

# 너무 계산 시간이 오래 걸리는 관계로 random sampling
people = people.sample(n=10000)
people.reset_index(inplace = True, drop = True)

# 좌표 설정 제대로 되지 않은 좌표 제거
drop_shelter = []

for i in range(shelter.shape[0]):
    if shelter.loc[i, "x"] == 0:
        drop_shelter.append(i)

shelter.drop(drop_shelter, inplace = True)
shelter.reset_index(drop = True, inplace = True)

# 0.15km 안에 위치한 경로당은 대표 하나만 묶기
li_drop = [] # 버릴 리스트 

shelter_lat = shelter['y']
shelter_long = shelter['x']

for j in range(shelter.shape[0]-1):
    li_group = []
    li_group.append(j)
    for i in range(j+1, shelter.shape[0]):
        start = (float(shelter_lat[j]), float(shelter_long[j]))
        goal = (float(shelter_lat[i]), float(shelter_long[i]))
        if haversine(start, goal) <= 0.15:
            li_group.append(i)
    if len(li_group) >= 2:
        li_drop.append(li_group)

# 각 대표 1개만 선정
li = []

for i in li_drop:
    li.append(random.choice(i))
    
li_drop_final = sum(li_drop, [])

a_sub_b = [x for x in li_drop_final if x not in li]

a_sub_b

shelter = shelter.drop(a_sub_b)
shelter.reset_index(drop = True, inplace = True)

In [7]:
print("사람 수 : " + str(len(people)))
print("시설 수 : " + str(len(shelter)))

사람 수 : 10000
시설 수 : 154


In [11]:
people_lat = people['y']
people_long = people['x']

shelter_lat = shelter['y']
shelter_long = shelter['x']

# 거리 구하기
c = []
for i in range(people.shape[0]):
    list_ = []
    for j in range(shelter.shape[0]):
        start = (float(people_lat[i]), float(people_long[i]))
        goal = (float(shelter_lat[j]), float(shelter_long[j]))
        list_.append(haversine(start, goal))
    c.append(list_)

In [ ]:
m = people.shape[0] # 노인인구 수
n = shelter.shape[0] # 스마트경로당 후보지 수
p = 12 # 찾을 시설의 수

# 문제 정의
prob = LpProblem("p-median", LpMinimize)

# 결정 변수 정의
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

- c : 각 시설별로 각 고객에게 서비스를 제공하는 비용(cost함수 정의 후 만들기)
- i행은 고객을 나타냄
- j열은 시설을 나타냄
- c[i][j] : i번째 고객에게 j번째 facility가 서비스를 제공하는 비용

- x는 이진 결정 변수의 dictionary를 정의
    - 각 변수는 i고객이 j시설에 할당되었는지 여부를 나타냄
    
- 각각의 시설이 열렸는지 닫혔는지?

In [14]:
# 목적함수 정의
prob += lpSum([c[i][j] * x[(i,j)] for i in range(m) for j in range(n)])

In [15]:
for i in range(m):
    prob += lpSum([x[(i,j)] for j in range(n)]) == 1
# 한 사람은(i번째 사람) 한 시설(j번째 시설)에만 영향을 받아야 함

for i in range(m):
    for j in range(n):
        prob += x[(i,j)] <= y[j]
# 시설이 열려 있어야지 영향을 받고 있는지 여부를 0과 1로 결정할 수 있음

prob += lpSum([y[j] for j in range(n)]) == p
# 시설이 열려있는 수는 우리가 원하는 시설의 수와 같아야됨

In [16]:
# branch-and-bound algorithm을 이용해 문제 해결하기
random.seed(123)
prob.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=10000))

1

- gapRel = 0.0 : 주어진 시간 제한 내에서 최상의 솔루션을 찾을 때까지 최적의 솔루션을 계속 검색
- threads = 1 : 병렬 컴퓨팅에 사용할 수 있는 thread 수를 의미, 단일 thread 모들에서 실행한다는 의미인 1로 실행
- timeLimit = 600 : 솔루션을 찾는 데 사용할 수 있는 최대 시간(초)

In [17]:
li_located = []

# Print results
print("Optimal objective value:", value(prob.objective))
for j in range(n):
    if y[j].value() > 0.5:
        li_located.append(j)
        print("Facility", j, "is located.")
#        for i in range(m):
#                if x[(i,j)].value() > 0.5:
#                    print("- Customer", i, "is served.")

Optimal objective value: 6383.34220193535
Facility 23 is located.
Facility 31 is located.
Facility 55 is located.
Facility 57 is located.
Facility 61 is located.
Facility 70 is located.
Facility 72 is located.
Facility 81 is located.
Facility 92 is located.
Facility 142 is located.
Facility 150 is located.
Facility 153 is located.


- 첫 번째 줄 : p개 시설을 open했을 때 최소 비용을 인쇄(목표 값)
- 그 다음 각각 시설에 대해 코드가 시설이 열렸는지 여부를 확인
    - y[j]의 값이 0.5보다 크면 j번째 시설이 열렸다는 의미

In [18]:
import folium
lat = shelter['y'].mean()
long = shelter['x'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 20)

#for n in range(shelter.shape[0]):
for n in li_located:
    popup = shelter.loc[n, '사업장명'],
    location = [shelter.loc[n, "y"], shelter.loc[n, "x"]]
    folium.Marker(
        location = location,
        popup = popup
    ).add_to(m)
m

In [19]:
df_result = shelter.loc[li_located]
df_result

,사업장명,소재지전체주소,x,y
23,중앙하이츠,서울특별시 강서구 화곡동 351-89번지,126.843685,37.535717
31,푸른 경로당,서울특별시 강서구 화곡동 1016-34번지,126.834776,37.544575
55,골안말 경로당,서울특별시 강서구 화곡동 865-1번지,126.856096,37.533251
57,방화12단지아파트 경로당,서울특별시 강서구 방화동 852번지,126.806639,37.569805
61,상사경로당,서울특별시 강서구 개화동 230-13번지,126.801563,37.586787
70,등촌5단지(아)경로당,서울특별시 강서구 등촌동 695번지,126.844869,37.560452
72,봉제경로당,서울특별시 강서구 화곡동 산 47-16번지,126.850910,37.545054
81,태영송화(아)경로당,서울특별시 강서구 염창동 270-1번지,126.867212,37.553378
92,가양8단지아파트경로당,서울특별시 강서구 가양동 1482번지,126.857174,37.562096
142,마곡수명산파크 7단지(아) 경로당,서울특별시 강서구 내발산동,126.821918,37.547663


## 이미 설치된 경로당으로부터 반경 특정 범위 안에 있는 사람들 지우고 12개 다시 선정
- 선행연구에서는 노인의 보행권의 거리를 최소 300, 최대 1km로 보고 있으며, 평균적으로 이동할 수 있는 거리를 500m로 제시하고 있음
    - Accessibility to Welfare Facilities for the Aged through GIS Network Analysis : Focused on Inland Areas in Incheon (Ma Sein 등, 2011)
- haversine distance : km단위
    - 직선거리이기 때문에 최소거리인 300m를 기준으로 잡아보기로 함 -> 0.3

In [ ]:
li_drop = [] # 버릴 리스트 

people_lat = people['y']
people_long = people['x']

shelter_lat = shelter.loc[li_located, 'y'].reset_index(drop = True)
shelter_long = shelter.loc[li_located, 'x'].reset_index(drop = True)

In [20]:
# 이미 설치된 경로당으로부터 0.3km 안에 있는 사람들 지우기
for j in range(shelter.loc[li_located, :].shape[0]):
    for i in range(people.shape[0]):
        start = (float(people_lat[i]), float(people_long[i]))
        goal = (float(shelter_lat[j]), float(shelter_long[j]))
        if haversine(start, goal) <= 0.3:
            li_drop.append(i)
            
len(li_drop)

[12,
 32,
 43,
 45,
 101,
 150,
 207,
 228,
 311,
 318,
 364,
 411,
 478,
 488,
 509,
 515,
 600,
 634,
 642,
 729,
 778,
 785,
 982,
 1038,
 1046,
 1064,
 1271,
 1305,
 1337,
 1412,
 1413,
 1434,
 1498,
 1562,
 1623,
 1651,
 1734,
 1817,
 1918,
 1926,
 2029,
 2102,
 2174,
 2248,
 2304,
 2374,
 2379,
 2387,
 2418,
 2443,
 2456,
 2610,
 2633,
 2644,
 2732,
 2747,
 2795,
 2820,
 2842,
 2914,
 2916,
 2927,
 3083,
 3088,
 3215,
 3224,
 3468,
 3495,
 3579,
 3588,
 3646,
 3662,
 3725,
 3756,
 3812,
 3821,
 3830,
 3834,
 3883,
 3911,
 3922,
 3941,
 4036,
 4050,
 4229,
 4313,
 4320,
 4323,
 4326,
 4341,
 4374,
 4403,
 4484,
 4503,
 4536,
 4624,
 4643,
 4648,
 4680,
 4681,
 4723,
 4780,
 4784,
 4819,
 4914,
 4967,
 4975,
 5085,
 5160,
 5165,
 5188,
 5198,
 5241,
 5244,
 5260,
 5327,
 5344,
 5345,
 5346,
 5377,
 5382,
 5434,
 5459,
 5542,
 5547,
 5636,
 5646,
 5652,
 5660,
 5708,
 5781,
 5802,
 5806,
 5814,
 5858,
 6002,
 6045,
 6216,
 6218,
 6255,
 6261,
 6304,
 6322,
 6342,
 6441,
 6635,
 6673

In [21]:
people.drop(li_drop, inplace = True)
people.reset_index(inplace = True, drop = True)

shelter.drop(li_located, inplace = True)
shelter.reset_index(inplace = True, drop = True)

In [ ]:
print("사람 수" : len(people))
print("시설 수" : len(shelter))

In [23]:
people_lat = people['y']
people_long = people['x']

shelter_lat = shelter['y']
shelter_long = shelter['x']

# 거리 함수를 바꿔야함(현재 haversine)
c = []
for i in range(people.shape[0]):
    list_ = []
    for j in range(shelter.shape[0]):
        start = (float(people_lat[i]), float(people_long[i]))
        goal = (float(shelter_lat[j]), float(shelter_long[j]))
        list_.append(haversine(start, goal))
    c.append(list_)
    
m = people.shape[0]
n = shelter.shape[0]
p = 12

# 문제 정의
prob = LpProblem("p-median", LpMinimize)

# 결정 변수 정의
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

# 목적함수 정의
prob += lpSum([c[i][j] * x[(i,j)] for i in range(m) for j in range(n)])

for i in range(m):
    prob += lpSum([x[(i,j)] for j in range(n)]) == 1
# 한 사람은(i번째 사람) 한 시설(j번째 시설)에만 영향을 받아야 함

for i in range(m):
    for j in range(n):
        prob += x[(i,j)] <= y[j]
# 시설이 열려 있어야지 영향을 받고 있는지 여부를 0과 1로 결정할 수 있음

prob += lpSum([y[j] for j in range(n)]) == p
# 시설이 열려있는 수는 우리가 원하는 시설의 수와 같아야됨

# branch-and-bound algorithm을 이용해 문제 해결하기
random.seed(123)
prob.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=10000))

1

In [24]:
li_located = []

# Print results
print("Optimal objective value:", value(prob.objective))
for j in range(n):
    if y[j].value() > 0.5:
        li_located.append(j)
        print("Facility", j, "is located.")
#        for i in range(m):
#                if x[(i,j)].value() > 0.5:
#                    print("- Customer", i, "is served.")

Optimal objective value: 5684.06172676554
Facility 4 is located.
Facility 26 is located.
Facility 31 is located.
Facility 40 is located.
Facility 62 is located.
Facility 75 is located.
Facility 76 is located.
Facility 80 is located.
Facility 97 is located.
Facility 118 is located.
Facility 131 is located.
Facility 133 is located.


In [25]:
import folium
lat = shelter['y'].mean()
long = shelter['x'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 20)

#for n in range(shelter.shape[0]):
for n in li_located:
    popup = shelter.loc[n, '사업장명'],
    location = [shelter.loc[n, "y"], shelter.loc[n, "x"]]
    folium.Marker(
        location = location,
        popup = popup
    ).add_to(m)
m

In [26]:
df_result = shelter.loc[li_located]
df_result

,사업장명,소재지전체주소,x,y
4,마곡엠밸리2단지아파트경로당,서울특별시 강서구 마곡동,126.821411,37.568825
26,봉바위경로당,서울특별시 강서구 화곡동 1061-36번지,126.841552,37.543125
31,동성아파트경로당,서울특별시 강서구 방화동 817번지,126.812182,37.579603
40,가양9-1단지아파트경로당,서울특별시 강서구 가양동 1489번지,126.861417,37.560124
62,연지경로당,서울특별시 강서구 화곡동 1121-19번지,126.852646,37.552416
75,강서한강자이아파트,서울특별시 강서구 가양동,126.848183,37.566328
76,구립예촌 경로당,서울특별시 강서구 화곡동 425-9번지,126.842126,37.530426
80,내발산 경로당,서울특별시 강서구 내발산동 671-30번지,126.840095,37.554193
97,신안빌라 경로당,서울특별시 강서구 가양동 327-53번지,126.853384,37.535847
118,마곡수명산파크 6단지(아) 경로당,서울특별시 강서구 내발산동 759번지 마곡수명산파크 6단지,126.825822,37.549875
